In [ ]:
from sympy import sympify, limit, oo

This is the `1s0 -> 1s0` and `3s1 -> 3s1`  transition

In [ ]:
ss = """
L ** 2 * (L ** 2 + k ** 2 / 4 - k * p_o - p_i ** 2 + p_o ** 2) * exp(
    -(p_i ** 2 + (k / 2 - p_o) ** 2) / L ** 2
) / (pi ** 2 * k * p_o) - L ** 2 * (
    L ** 2 + k ** 2 / 4 + k * p_o - p_i ** 2 + p_o ** 2
) * exp(
    -(p_i ** 2 + (k / 2 + p_o) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_o
) + L ** 2 * (
    L ** 2 + k ** 2 / 4 - k * p_i + p_i ** 2 - p_o ** 2
) * exp(
    2 * k * p_i / L ** 2 - (p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_i
) - L ** 2 * (
    L ** 2 + k ** 2 / 4 + k * p_i + p_i ** 2 - p_o ** 2
) * exp(
    -(p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_i
)
"""
ss = sympify(ss)

In [ ]:
ss

In [ ]:
limit(ss, "L", oo)

This is the `3s1 -> 3d1` transition Apparently needs a factor of two from `d->s`.

In [ ]:
sd = """
2 * sqrt(2) * L ** 2 * p_o ** 2 * exp(-(p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2) / (
    3 * pi ** 2 * k * p_i
) - 2 * sqrt(2) * L ** 2 * p_o ** 2 * exp(-(p_o ** 2 + (k / 2 - p_i) ** 2) / L ** 2) / (
    3 * pi ** 2 * k * p_i
) + (
    -sqrt(2) * L ** 6 / (2 * pi ** 2 * k * p_o ** 3)
    - sqrt(2) * L ** 4 / (2 * pi ** 2 * p_o ** 2)
    - 2 * sqrt(2) * L ** 4 / (3 * pi ** 2 * k * p_o)
    - sqrt(2) * L ** 2 * k / (6 * pi ** 2 * p_o)
    - 2 * sqrt(2) * L ** 2 / (3 * pi ** 2)
    - 2 * sqrt(2) * L ** 2 * p_o / (3 * pi ** 2 * k)
) * exp(
    -(p_i ** 2 + (k / 2 + p_o) ** 2) / L ** 2
) + (
    sqrt(2) * L ** 6 / (2 * pi ** 2 * k * p_o ** 3)
    - sqrt(2) * L ** 4 / (2 * pi ** 2 * p_o ** 2)
    + 2 * sqrt(2) * L ** 4 / (3 * pi ** 2 * k * p_o)
    + sqrt(2) * L ** 2 * k / (6 * pi ** 2 * p_o)
    - 2 * sqrt(2) * L ** 2 / (3 * pi ** 2)
    + 2 * sqrt(2) * L ** 2 * p_o / (3 * pi ** 2 * k)
) * exp(
    -(p_i ** 2 + (k / 2 - p_o) ** 2) / L ** 2
)
"""
sd = sympify(sd)

In [ ]:
ds = sd.subs({"p_i": "p_o", "p_o": "p_i"}) * 2

In [ ]:
str(ds)

In [ ]:
sd.simplify()

In [ ]:
limit(sd, "L", oo)

In [ ]:
from numpwd.qchannels.lsj import get_two_n_channels, generate_matrix_channels
from pandas import DataFrame

In [ ]:
channels = DataFrame(get_two_n_channels(l_max=2, s_max=1))
allowed = {
    "1s0": "s==l==j==0",
    "3s1": "(s==j==1 and l==0)",
    "3d1": "(s==j==1 and l==2)",
}

channels = channels.query(" or ".join(allowed.values())).reset_index()
channels

In [ ]:
allowed_transitions = [
    lambda ci, co: ci["ml"] == co["ml"]
    and ci["ms"] == co["ms"]
    and ci["mj"] == co["mj"]
    and ci["j"] == co["j"]
    and ci["s"] == co["s"]
    and ci["l"] in [0, 2]
    and co["l"] in [0, 2]
    and not (ci["l"] == co["l"] == 2)
]
df = (
    DataFrame(
        list(
            generate_matrix_channels(
                channels.to_dict("records"), allowed_transitions=allowed_transitions
            )
        )
    )
    .drop(
        columns=["cg_o", "cg_i", "index_o", "index_i", "ml_o", "ml_i", "ms_o", "ms_i"]
    )
    .groupby(["l_o", "s_i", "j_i", "l_i", "s_o", "j_o", "mj_i", "mj_o"])
    .sum()
    .reset_index()
)
df

## Convert to operator format

In [ ]:
from numpwd.integrate.numeric import ExpressionMap
import numpy as np

In [ ]:
cutoff = 100
fcn_ss = ExpressionMap(ss.subs({"L": cutoff}), ["p_o", "p_i", "k"])
fcn_sd = ExpressionMap(sd.subs({"L": cutoff}), ["p_o", "p_i", "k"])
fcn_ds = ExpressionMap(ds.subs({"L": cutoff}), ["p_o", "p_i", "k"])


p = np.arange(1, 3)
k = np.array([2])

matrix_ss = fcn_ss(p, p, k)
matrix_sd = fcn_sd(p, p, k)
matrix_ds = fcn_ds(p, p, k)

In [ ]:
matrix = []
for idx, channel in df.iterrows():
    lo = "s" if channel["l_o"] == 0 else "d"
    li = "s" if channel["l_i"] == 0 else "d"
    so = str(2 * channel["s_o"] + 1)
    si = str(2 * channel["s_i"] + 1)
    jo = str(channel["j_o"])
    ji = str(channel["j_i"])

    ci = si + li + ji
    co = so + lo + jo

    if ci == co == "1s0" or ci == co == "3s1":
        matrix.append(matrix_ss * channel["cg"])
    elif ci == "3s1" and co == "3d1":
        matrix.append(matrix_sd * channel["cg"])
    elif ci == "3d1" and co == "3s1":
        matrix.append(matrix_ds * channel["cg"])
    else:
        raise ValueError(f"ci={ci}, co={co}")

matrix = np.array(matrix)

In [ ]:
from numpwd.operators.base import Operator

In [ ]:
op = Operator()
op.channels = df.drop(columns="cg")
op.args = [("p_o", p), ("p_i", p), ("q", k)]
op.isospin = {(0, 0, 0, 0): 1, (1, 1, 1, 1): 1, (1, 0, 1, 0): 1, (1, -1, 1, -1): 1}
op.matrix = matrix
op.misc["cutoff"] = cutoff
op.misc["pwd type"] = "analytic"
op.check()

In [ ]:
op.misc